<a href="https://colab.research.google.com/github/afolabiope/pagegit/blob/main/Page_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def get_data(
        ticker,
        start_date_str,
        end_date_str,
        multiplier,
        timeframe,
        trading_hours='RTH',
        interval_days=5):
        try:
            client = RESTClient(api_key='jWRu6HJz1LvzFczTFH4WRaaayBtAo6FG')
            ticker = ticker.upper()

            start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
            end_date = datetime.strptime(end_date_str, "%Y-%m-%d")

            aggs = []
            for a in client.list_aggs(
                ticker=ticker,
                multiplier=multiplier,
                timespan=timeframe,
                from_=start_date.strftime("%Y-%m-%d"),
                to=end_date.strftime("%Y-%m-%d"),
                limit=50000,
            ):
                aggs.append(a)

            for agg in aggs:
                timestamp_ms = agg.timestamp
                timestamp_seconds = timestamp_ms / 1000
                date_object = datetime.utcfromtimestamp(timestamp_seconds) - timedelta(hours=5)
                formatted_date = date_object.strftime('%Y-%m-%d %H:%M:%S')
                agg.timestamp = formatted_date

            data = []
            for agg in aggs:
                data.append({
                    'open': agg.open,
                    'high': agg.high,
                    'low': agg.low,
                    'close': agg.close,
                    'volume': agg.volume,
                    'vwap': agg.vwap,
                    'timestamp': agg.timestamp,
                    'transactions': agg.transactions,
                    'otc': agg.otc
                })

            df = pd.DataFrame(data)
            df['timestamp'] = pd.to_datetime(df['timestamp'])

            us_eastern = pytz.timezone('US/Eastern')

            def is_dst(dt):
                dst_start = datetime(dt.year, 3, 8)
                dst_end = datetime(dt.year, 11, 1)
                dst_start += timedelta(days=(6 - dst_start.weekday()) % 7)
                dst_end += timedelta(days=(6 - dst_end.weekday()) % 7)
                dst_start = us_eastern.localize(dst_start)
                dst_end = us_eastern.localize(dst_end)
                dt = dt.astimezone(us_eastern)
                return dst_start <= dt < dst_end

            df['timestamp'] = pd.to_datetime(df['timestamp']).dt.tz_localize(us_eastern)
            df['timestamp'] = df['timestamp'].apply(lambda x: x + timedelta(hours=1) if is_dst(x) else x)

            if trading_hours == 'RTH':
                rth_start_time = pd.to_datetime("09:30:00").time()
                rth_end_time = pd.to_datetime("15:59:59").time()

                df = df[(df['timestamp'].dt.time >= rth_start_time) & (df['timestamp'].dt.time <= rth_end_time)]

                df = df.reset_index(drop=True)

            df['timestamp'] = pd.to_datetime(df['timestamp']).dt.tz_convert(us_eastern).dt.tz_localize(None)

            result_dict = df.to_dict(orient='list')
            # print("result_dict", result_dict)
            return result_dict

        except Exception as e:
            # Log the error for debugging
            print(f"Error in get_data: {e}")
            # Provide a user-friendly error message in the response
            return {"error": "An unexpected error occurred while fetching data."}


In [1]:
!pip install polygon-api-client==1.13.4

In [2]:
from polygon import RESTClient